In [1]:
import sys, os, re, copy
import dill as pickle # this serializes all the functions inside the quantification dict
import numpy as np
import scipy as sp
from scipy.optimize import newton, minimize, fsolve
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
import matplotlib.colors as mcolors
palette = list(mcolors.TABLEAU_COLORS.keys())
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid", {
 'axes.spines.bottom': True,
 'axes.spines.left': True,
 'axes.spines.right': True,
 'axes.spines.top': True
})
sns.set(font_scale=1)
palette = list(mcolors.TABLEAU_COLORS.keys())
sns.set_theme(style="ticks", palette="muted")
sns.set_context("talk")

## Acid hydrolysis based quantification of total amino acids and nucleotides
This notebook works through the quantification of total amino acids and nucleotides in 143B and H1299 cell lines.
Total amino acids and nucleotides should be understood as free amino acids + amino acids bound in proteins/peptides and free nucleotides/nucleosides/nucleobases + nucleotides bound in DNA/RNA.
To generate cell material for acid hydrolysis the following was setup:
* Prior to seeding the experiment, cells had been grown 20 doublings in media with the isotopic labels (U-13C Asn and 15N-amide Gln) used in the experiment. Thus, the fractional labeling was a steady-state at the start of the experiment.
* 3x 6-well plates for each cell line (143B and H1299)
* Seeded at ~100,000 cells/well in DMEM, without pyruvate and glutamine, with 10% dialyzed FBS and 1x penstrep
* 15N-amide Gln was spiked in at 4 mM
* On two plates 100 μM U-13C Asn was spiked in, on one plate 1 mM U-13C Asn was spiked in
* At 50-80% confluency extracted cell material from wells in the top row and counted cells from the wells in the bottom row
* To extract cell material first washed twice with cold saline, the added 1 mL 80% MeOH, scraped cells and transferred material to a clean tube. Then washed each well with 4x 1 mL water to remove any residual cell material.
* The bottom row was counted on a Coulter counter to find the cell count and average cell volume

At this point the cell material was thoroughly mixed and transferred in 1 mL aliquots to 1.5 mL tubes after which the acid hydrolysis protocol began.
Tubes were dried and two replicates were set aside for fast, low temperature acid hydrolysis for nucleobase quantification, two replicates were set aside for slow, high temperature acid hydrolysis for amino acid quantification and one tube was set aside for determining the ratio of labelled vs unlabelled Asn i.e. no acid hydrolysis.
To the acid hydrolysis samples 150 μL 6 M HCl (amino acid analysis quality) was added and nucleobase quantification samples were hydrolyzed at 80C for 2 hours and amino acid quantification samples were hydrolyzed at 90C for 72 hours.
After acid hydrolysis tubes were dried.
Then 1 mL 80% MeOH was added to all tubes, they were thoroughly mixed, spun down and 800 μL was transferred to fresh tubes for the non-hydrolysis samples, 50 μL for nucleobase samples and 20 μL for amino acid samples.
They were all dried and non-hydrolysis samples were reconstituted in 40 μL 80% MeOH whereas acid hydrolysis samples were reconstituted in 40 μL 80% MeOH with U-13C internal standards.
Amino acid quantification samples were also run without U-13C internal standards to determine m+0/m+4 for aspartate as this quantifies the contribution of hydrolyzed asparagine to the measured aspartate concentration.
Finally, the content of each tube was transferred to LCMS vials and quantified.


#### Notes on the use of isotopic labels
The use of U-13C Asn is to label a large fraction of the asparagine that gets incorporated into protein.
Thus, when acid hydrolysis is perform and all the asparagine is deaminated into aspartate the a two can be deconvoluted by using the label.
This does present a slight problem though, as the quantification is also performed by using U-13C internal standards.
I deal with this issue below.

The experiment is also using 15N-amide Gln which was added to account for glycine derived from purine breakdown duirng acid hydrolysis; however, I later found out that I had made a mistake and that the amide nitrogen on glutamine does not label the glycine that gets released from purines.
In fact this glycine from purine breakdown is the same glycine that get incorporatd during purine synthesis, so there is no way of discriminating between the glycine in protein and glycine from purine breakdown.
While to usefull for labelling glycine, the use of 15N-amide Gln does present a slight nuisance in that it introduces a natural abundance error in the nucleobases/nucleosides.
Luckily, the isotopic enrichment of the 15N-amide Gln is almost identical to the natural isotope abundance of 14N, thus cancelling out the natural abundance error.
I still account for the error though, but it only changes the concentrations by a few percent.


#### Limitations to this type of quantification
While LCMS and isotopic internal standards are really good for quantification, there are some important limitations related to undercounting of modified amino acids and nucleosides/nucleobases.
Some abundance amino acid modifiations like phosphorylation, acetylation and glycosylation are cleaved under acid hydrolysis but others like methylation are not.
The same is true for nucleobases were methylation would be expected to cause an underestimate of certain bases.

In [2]:
def impute_conc(piece_wise_fit_metab, response_ratio):
    '''
    This function imputes the concentration from a response ratio.
    '''
    response_ratio_range = np.array(list(piece_wise_fit_metab.keys()))
    mask_range = [response_ratio >= min_v and response_ratio <= max_v for max_v, min_v in response_ratio_range]
    k = tuple(response_ratio_range[mask_range][0])
    conc = piece_wise_fit_metab[k](response_ratio)
    estimator = 'intrapolation'
    if 0 in k:
        estimator = 'extrapolation under'
    elif np.inf in k:
        estimator = 'extrapolation over'
    return(conc, estimator)

In [3]:
def NB_ratio_nitrogen(Npos, label_abundance_15N, natural_abundance_14N=0.99636):
    '''
    Calculates a multiplier to correct concentration
    of nitrogen labelled compounds.
    Multiply the return value with the calculated concentration
    to perform the correction.
    Npos = positions that can be labelled
    '''
    # Under natural abundance
    # what is the expected fraction of the compound
    # with _no_ 15N on all the positions that could be labelled:
    frac_natural = sp.stats.binom.pmf(Npos, Npos, natural_abundance_14N)
    # With the label
    # what is the expected fraction of the compound
    # with 15N on all the positions that could be labelled:
    frac_label = sp.stats.binom.pmf(Npos, Npos, label_abundance_15N)
    return(frac_natural/frac_label)

#### Correction for aspartate
Aspartate is a special case for quantification because U-13C labelled asparagine was feed to cell to descriminate protein levels of asparagine and aspartate since protein bound U-13C asparagine turns into U-13C aspartate upon acid hydrolysis; however, the U-13C aspartate is also part of the internal standards for quantification. Thus the U-13C aspartate coming from protein U-13C asparagine has to be decoupled from U-13C aspartate coming from the internal standard. This is achieved by finding the ratio a unlabelled to labelled aspartate, by running samples without internal standards, and then correcting the response ratio accordingly.

Let $A_1$ be the peak area of unlabelled aspartate, $A_2$ be that of U-13C aspartate derived from hydrolyzed U-13C asparagine and $N$ be that derived from the internal standard. We then have:
$$
Y = \frac{A_1}{A_2+N} => N = \frac{A_1}{Y-A_2}
$$

From the ratio we have:
$$
X = \frac{A_1}{A_2} => A_2 = \frac{A_1}{X}
$$

By substitution we get:
$$
N = \frac{A_1}{Y}-\frac{A_1}{X}
$$


And the corrected response ratio ($RR$) for finding the concentration of protein aspartate and asparagine can then be found:
$$
RR = \frac{A_1+A_2}{N} => \frac{A_1+\frac{A_1}{X}}{\frac{A_1}{Y}-\frac{A_1}{X}} => \frac{X+1}{\frac{X}{Y}-1}
$$

The response ratio is converted to a concentration of total aspartate through the calibration curve $f(RR)$. Then the fraction of aspartate from hydrolyzed asparagine is calculated using the aspartate label fraction $X$. Incomplete labelling with asparagine is taken into account by using the fraction of labelled to unlabelled asparagine before acid hydrolysis ($F$). It is not necessary to correct for natural isotope abundance because the 13C enrichment in the U-13C asparagine is approximately equal to the natural 12C abundance. Thus, the final concentration will be:
$$
[Asn] = f(RR) \frac{1}{F(1+X)}
$$

$$
[Asp] = f(RR) (1 - \frac{1}{F(1+X)})
$$




In [4]:
def asp_corr(X, Y, F, f):
    '''
    Calculates the aspartate and asparagine concentration
    based on the above shown correction.
    
    Here is some code I used for testing:
    f_conc = lambda x: x*10
    F = 0.98 # labelling fraction
    A1 = 500+700*(1-F)
    A2 = 700*F
    N = 1000
    X = A1/A2
    Y = A1/(A2+N)
    RR = (X+1)/(X/Y-1)

    Asp_total = f_conc(RR)
    Asn_conc = Asp_total*   1/(F*(X+1))
    Asp_conc = Asp_total*(1-1/(F*(X+1)))

    print(Asp_conc)
    print(Asn_conc)
    assert((Asp_conc+Asn_conc) == Asp_total)
    '''
    RR = (X+1)/(X/Y-1)
    if RR < 0:
        return(0, 0, 'extrapolation under')
    else:
        asp_total, estimator = impute_conc(f, RR)
        asn_conc = asp_total*   1/(F*(X+1))
        asp_conc = asp_total*(1-1/(F*(X+1)))
        return(asp_conc, asn_conc, estimator)

Final objective: show Asp fate flux from different experiments

Also plot data relating to total protein.
Use "143B-H1299_Metformin" data to quantify amino acids


In [5]:
# Dict to correct for nitrogen natural isotope abundance:
NB_correct_dict = {
    'Guanine 15N3 pos': 3,
    'Adenine 15N2 pos': 2,
    'Uracil 15N1 neg': 1,
    'Cytosine 15N2 pos': 2,
    'Thymine 15N1 neg': 1,
    'Hypoxanthine 15N2 pos': 2,
    'Xanthine 15N2 neg': 2,
    'Guanosine 15N3 pos': 3,
    'Adenosine 15N2 pos': 2,
    'Uridine 15N1 neg': 1,
    'Cytidine 15N2 pos': 2,
    'Deoxyuridine 15N1 neg': 1,
}
# List of amino acids with quantification data:
AA_quant_list = ['Asparagine', 'Aspartate neg', 'Glutamate neg', 'Arginine pos', 'Tyrosine pos', 'Serine neg', 'Proline pos', 'Threonine neg', 'Lysine pos', 'Leucine pos', 'Valine pos', 'Phenylalanine pos', 'Isoleucine pos', 'Alanine pos']

In [6]:
### Read quantification function ###
dict_pickle_fnam = 'AA-nucleoside_quant-dict.pickle'
with open(dict_pickle_fnam, 'rb') as handle:
    piece_wise_fit_AA_nucs = pickle.load(handle)
# dict_pickle_fnam = 'AA-nucleobase_quant-dict.pickle'
dict_pickle_fnam = 'AA-nucleoside_quant-dict.pickle' # change to above
with open(dict_pickle_fnam, 'rb') as handle:
    piece_wise_fit_nucb = pickle.load(handle)
# Merge the two quantitation dictionaries:
piece_wise_fit_quant = {**piece_wise_fit_AA_nucs, **piece_wise_fit_nucb}

### Read measurements ###
### Replace all N/F with 0 before start ###
esheet_dict_mes = pd.read_excel('143B-H1299_Nuc-Prot_quant.xlsx', sheet_name=None)
annotation_df = pd.read_excel('annotations.xlsx')
metab_dict_mes = dict()
metab_names_mes = list()
for k in esheet_dict_mes.keys():
    if 'U-13C' not in k:
        metab_names_mes.append(k)
        metab_dict_mes[k] = copy.deepcopy(esheet_dict_mes[k])
        metab_dict_mes[k]['Response Ratio'] = metab_dict_mes[k]['Area'] / metab_dict_mes[k]['ISTD Response']
        metab_dict_mes[k]['Response Ratio'] = metab_dict_mes[k]['Response Ratio'].fillna(0).replace(np.inf, 0)
        metab_dict_mes[k]['Sample_name'] = [fn.split('_')[-1] for fn in metab_dict_mes[k]['Filename']]
        # Add annotations:
        metab_dict_mes[k] = metab_dict_mes[k].merge(annotation_df, left_on='Sample_name', right_on='Sample_name')
        if 'Aspartate' in k:
            metab_dict_mes[k] = metab_dict_mes[k].drop(['Flag Details', 'Theoretical Amt', 'Filename', 'Type', 'RT', 'Sample ID'], axis=1)
        else:
            metab_dict_mes[k] = metab_dict_mes[k].drop(['Flag Details', 'Theoretical Amt', 'Filename', 'Type', 'RT', 'Sample ID', 'Asp_X', 'Asn_frac'], axis=1)

### Impute concentration and add to metabolite dataframe ###
rr_mes = dict() # for plotting 
imp_conc_mes = dict() # for plotting
for metab in metab_names_mes[:]:
    # Remove 15N from name:
    metab_split = metab.split()
    if len(metab_split) == 4:
        metab_no_iso = ' '.join([metab_split[0], metab_split[1], metab_split[-1]])
    elif len(metab_split) == 3:
        metab_no_iso = ' '.join([metab_split[0], metab_split[-1]])
    elif len(metab_split) == 2:
        metab_no_iso = metab
    else:
        raise Exception('{} not recognized metabolite name format'.format(metab))
    # Assign imputed concentration:
    if metab_no_iso in piece_wise_fit_quant:
        if 'Aspartate' not in metab_no_iso: # special case for aspartate
            conc_list = list()
            estimator_list = list()
            for label_abundance_15N, rr in zip(metab_dict_mes[metab]['Gln_frac'].values, metab_dict_mes[metab]['Response Ratio'].values):
                conc, estimator = impute_conc(piece_wise_fit_quant[metab_no_iso], rr)
                if metab in NB_correct_dict:
                    Npos = NB_correct_dict[metab]
                    NB_mult = NB_ratio_nitrogen(Npos, label_abundance_15N)
                else:
                    NB_mult = 1
                conc_list.append(NB_mult*conc)
                estimator_list.append(estimator)
            metab_dict_mes[metab]['imputed_sample_conc'] = conc_list
            metab_dict_mes[metab]['imputed_sample_estimator'] = estimator_list
        else:
            asp_conc_list = list()
            asn_conc_list = list()
            estimator_list = list()
            for X, Y, F in zip(metab_dict_mes[metab]['Asp_X'].values, metab_dict_mes[metab]['Response Ratio'].values, metab_dict_mes[metab]['Asn_frac'].values):
                asp_conc, asn_conc, estimator = asp_corr(X, Y, F, piece_wise_fit_quant[metab_no_iso])  ###
                asp_conc_list.append(asp_conc)
                asn_conc_list.append(asn_conc)
                estimator_list.append(estimator)
            metab_dict_mes['Asparagine'] = copy.deepcopy(metab_dict_mes[metab])
            metab_names_mes.append('Asparagine')
            metab_dict_mes[metab]['imputed_sample_conc'] = asp_conc_list
            metab_dict_mes[metab]['imputed_sample_estimator'] = estimator_list
            metab_dict_mes['Asparagine']['imputed_sample_conc'] = asn_conc_list
            metab_dict_mes['Asparagine']['imputed_sample_estimator'] = estimator_list

        # Extract data for plotting:
        if metab_no_iso in rr_mes:
            rr_mes[metab_no_iso].extend(list(metab_dict_mes[metab]['Response Ratio'].values))
            imp_conc_mes[metab_no_iso].extend(list(metab_dict_mes[metab]['imputed_sample_conc'].values))
        else:
            rr_mes[metab_no_iso] = list(metab_dict_mes[metab]['Response Ratio'].values)
            imp_conc_mes[metab_no_iso] = list(metab_dict_mes[metab]['imputed_sample_conc'].values)

In [7]:
metab_dict_mes[metab_names_mes[0]].drop(['Compound', 'Formula', 'Response Ratio', 'Area', 'ISTD Response'], axis=1)

,Sample_name,cell_line,plate,replicate,hydrolysis,dilution,cell_numb,cell_vol,frac_transferred,solvent_vol,Gln_frac
0,Q01,143B,1,1,none,1.00,472133.333333,2.517313,0.25,1000,0.985233
1,Q02,143B,1,1,short,1.25,472133.333333,2.517313,0.25,1000,0.985233
2,Q03,143B,1,1,short,1.25,472133.333333,2.517313,0.25,1000,0.985233
3,Q04,143B,1,1,long,0.50,472133.333333,2.517313,0.25,1000,0.985233
4,Q05,143B,1,1,long,0.50,472133.333333,2.517313,0.25,1000,0.985233
...,...,...,...,...,...,...,...,...,...,...,...
85,Q86,H1299,3,3,none,1.00,766466.666667,5.031903,0.25,1000,0.995494
86,Q87,H1299,3,3,short,1.25,766466.666667,5.031903,0.25,1000,0.995494
87,Q88,H1299,3,3,short,1.25,766466.666667,5.031903,0.25,1000,0.995494
88,Q89,H1299,3,3,long,0.50,766466.666667,5.031903,0.25,1000,0.995494


In [15]:
df = copy.deepcopy(metab_dict_mes[metab_names_mes[0]]).drop(['Compound', 'Formula', 'Response Ratio', 'Area', 'ISTD Response'], axis=1)
for nam in metab_names_mes:
    if nam in AA_quant_list:
        assert(metab_dict_mes[nam]['Sample_name'].equals(df['Sample_name']))
        df[nam] = metab_dict_mes[nam]['imputed_sample_conc']

mask = df['hydrolysis'] != 'none'
df = df[mask]
df_mean = df.groupby(['cell_line', 'plate', 'replicate', 'hydrolysis']).mean().reset_index()


In [16]:
mask = df_mean['hydrolysis'] == 'long'
df_mean_cell_conc = copy.deepcopy(df_mean[mask])
df_mean_cell_conc.loc[:, AA_quant_list] = df_mean_cell_conc.loc[:, AA_quant_list].mul( df_mean_cell_conc['solvent_vol'] / (df_mean_cell_conc['dilution'] * df_mean_cell_conc['frac_transferred'] * df_mean_cell_conc['cell_vol']), axis=0)


In [17]:
df_mean_cell_conc.loc[:, AA_quant_list].sum(axis=1)

0     405847.172563
2     305643.846359
4     418154.094452
6     473372.158272
8     550803.638019
10    388791.883413
12    426735.965852
14    432118.166791
16    414773.613990
18    353803.961388
20    332043.046867
22    345929.567945
24    376711.684291
26    357100.404298
28    376019.638340
30    278354.933249
32    384704.475325
34    352085.359395
dtype: float64

In [11]:
df_mean_cell_conc.drop(['hydrolysis', 'dilution', 'frac_transferred', 'solvent_vol'], axis=1).to_excel("conc.xlsx")

In [18]:
df_mean_cell_conc.loc[:, AA_quant_list]

,Asparagine,Aspartate neg,Glutamate neg,Arginine pos,Tyrosine pos,Serine neg,Proline pos,Threonine neg,Lysine pos,Leucine pos,Valine pos,Phenylalanine pos,Isoleucine pos,Alanine pos
0,14209.641058,33960.428148,91324.789059,23099.650440,9013.009872,29574.282455,22447.863290,26401.966416,34558.063813,37810.172983,15868.232167,15564.218859,15916.622931,36098.231073
2,10547.786654,25489.102193,77495.038071,16261.827977,6652.724237,22394.453884,16880.379661,19589.161877,24565.953482,27536.975856,9630.635839,11077.034432,10167.105908,27355.666287
4,14426.544429,34311.183600,92310.915183,23925.852594,9161.662291,32827.279367,22905.081604,27692.644020,35666.043185,38796.797152,16749.258397,16011.316602,16889.914987,36479.601040
6,17082.430854,41098.613836,104947.756431,26903.889565,11245.042137,37079.509959,27427.783587,29722.012267,40148.577269,43493.393212,15853.355158,17808.819919,15959.345602,44601.628477
8,19207.228326,45968.467151,117953.422335,31884.968717,12529.260920,43980.718541,31144.262566,35157.943265,47567.273882,51868.382575,22243.157138,21054.740639,20643.168943,49600.643022
10,13146.119191,31910.640989,93736.304049,21307.965502,9710.615324,31173.019732,21253.331553,24999.246834,31578.995111,35334.994101,13168.852436,14436.841496,13405.772926,33629.184167
12,26992.165606,28355.780026,86585.440578,25232.951509,10779.369470,33306.814171,22756.478260,26997.590207,36770.694127,39208.899272,18691.720826,16297.394028,17720.601664,37040.066109
14,25379.201311,28128.108482,87129.660312,25944.788596,11002.841942,33519.444762,23379.394886,27546.969815,37617.306087,40273.662007,19334.972150,16651.242071,17860.890946,38349.683425
16,25681.511556,27916.425707,84645.522743,24337.392149,10338.038214,31995.051576,22442.730894,26010.845501,35677.002616,38381.999397,18170.137758,15751.378788,16454.775102,36970.801991
18,13764.837823,29775.666494,74235.783535,20906.819729,8746.970943,31240.582923,18773.885135,23763.716490,30691.868384,34038.824885,12337.589134,14320.940643,12246.607825,28959.867447


In [14]:
df

,Sample_name,cell_line,plate,replicate,hydrolysis,dilution,cell_numb,cell_vol,frac_transferred,solvent_vol,...,Isoleucine pos,Leucine pos,Lysine pos,Phenylalanine pos,Proline pos,Serine neg,Threonine neg,Tyrosine pos,Valine pos,Asparagine
1,Q02,143B,1,1,short,1.25,472133.333333,2.517313,0.25,1000,...,1.517451,5.320199,5.322044,2.643093,3.912430,8.385738,5.702190,2.263950,1.287215,3.706475
2,Q03,143B,1,1,short,1.25,472133.333333,2.517313,0.25,1000,...,1.331630,4.085621,3.769849,2.038697,3.001588,6.667038,5.156003,1.771426,1.070352,2.914837
3,Q04,143B,1,1,long,0.50,472133.333333,2.517313,0.25,1000,...,6.597055,14.844237,13.706800,6.132082,8.678572,11.163231,10.207546,3.432092,6.696851,5.449280
4,Q05,143B,1,1,long,0.50,472133.333333,2.517313,0.25,1000,...,3.419727,8.950776,8.041568,3.662922,5.448504,7.448702,6.407959,2.240050,3.289476,3.493249
6,Q07,143B,1,2,short,1.25,472133.333333,2.517313,0.25,1000,...,1.400197,4.924400,4.560485,2.371095,3.520269,7.849729,5.495986,2.014566,1.459988,3.334563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,Q85,H1299,3,2,long,0.50,766466.666667,5.031903,0.25,1000,...,10.616081,23.101006,20.462307,9.548794,11.782837,18.633604,15.306115,6.358343,11.851662,15.203555
86,Q87,H1299,3,3,short,1.25,766466.666667,5.031903,0.25,1000,...,2.251796,8.490978,8.123038,4.223976,5.744550,12.891382,7.448107,3.498249,1.904004,9.612010
87,Q88,H1299,3,3,short,1.25,766466.666667,5.031903,0.25,1000,...,2.218868,9.044742,8.551747,4.338353,5.959675,13.554271,7.639513,3.576994,1.618866,10.029306
88,Q89,H1299,3,3,long,0.50,766466.666667,5.031903,0.25,1000,...,8.411365,22.254834,19.822771,9.112954,11.531060,20.063850,14.488549,5.537725,9.856344,15.847205
